In [1]:
import os, sys
path='../'
print(path in sys.path)
if not path in sys.path:
    sys.path.append(path)

False


In [2]:
import pandas as pd
import pyutils.gdrive as gdr

In [24]:
# header = ['time_stamp', 'first_name', 'last_name', 'email', 'ssh_key']


def get_gsheet_df(gid, sheetname='Sheet0'):
    gst = gdr.gsheet(sheetid=gid,fauth='admin-10ac-service.json') 
    return gst.get_sheet_df(sheetname)
    

try:
    gsid = '1Vn3HaH9JXpSR4L9DCpcSambNBxMZfME5I6Dz9bgwXZQ'
    sheetname = 'Form responses 1'
    dfb6 = get_gsheet_df(gsid, sheetname=sheetname).T
except:
    raise
    #dfb6 = pd.read_csv('b6_trainees_public_ssh_key.csv')

****** using service account: /Users/yabebal/.credentials/admin-10ac-service.json
service account is being used without delegation..


In [25]:
print(dfb6.shape)
dfb6.head()

(32, 6)


,Timestamp,Email address,First Name,Last Name,Group Number,"Copy the content of your PUBLIC SSH KEY (open the public key, copy the content, and paste it here)"
1,23/01/2024 19:00:09,natrix277@gmail.com,Natnael,Melese,Tutor,ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDAzzcL/d...
2,23/01/2024 19:01:05,odolbiniyam@gmail.com,Biniyam,Ajaw,G1,ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQC6daFppt...
3,23/01/2024 19:06:56,azizadx.tech@gmail.com,Nasrallah,Hassan,G6,ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDWYFvJYP...
4,23/01/2024 19:10:16,abelbekele.addise@gmail.com,Abel,Bekele,G3,ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDMv5o9kX...
5,23/01/2024 19:26:31,mekmeskassa@gmail.com,mekdes,mesfin,G2,ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDJFlEw1y...


In [28]:
dfb6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 1 to 32
Data columns (total 6 columns):
 #   Column                                                                                               Non-Null Count  Dtype 
---  ------                                                                                               --------------  ----- 
 0   Timestamp                                                                                            32 non-null     object
 1   Email address                                                                                        32 non-null     object
 2   First Name                                                                                           32 non-null     object
 3   Last Name                                                                                            32 non-null     object
 4   Group Number                                                                                         32 non-null     ob

In [83]:
# iterate through each record of dataframe and create a durectory inside ./credentials using df.first_name and 
#df.last_name then create a file inside the directory with the name of df.first_name and df.last_name and 
#write the content of df.ssh_key inside the file
import os

prefix = "cohort-a"
colmap = {'ssh_key':'Copy the content of your PUBLIC SSH KEY (open the public key, copy the content,  and paste it here)'}

def create_user_file(df, week=7):
    nokrow = 0
    dfg = df.groupby('Group Number')
    for group, gdf in dfg:  
        print('group', group)
        gdf = gdf.fillna('')
        
        def func2(row):
            return f"{len(row), type(row)}"
        
        def func(row):
            #x, y = row['First Name'], row['Last Name']
            x, y = row[0], row[1]
            
            try:
                x = x.lower().strip().split()[0]
            except:
                x = ""
            try:
                y = y.lower().strip().split()[0]
            except:
                y = ""
                
            u = ""
            if x:
                u += x
            if y:
                u += "_" + y
                
            if x or y:
                return u
            else:
                return pd.nan
                        
        df_user_name = pd.DataFrame()        
        df_user_name['user_name'] = gdf[['First Name','Last Name']].apply(lambda row:  func(row), axis=1).dropna()     
        df_user_name['access'] = 'root'
        print(df_user_name)
        
        #create folder and file
        file_name = f'/Users/yabebal/My-Works/10academy/Codes/aws_bash/compute/user/{prefix}-w{week}-{group.lower()}.txt'
        print('writing to file', file_name)
        df_user_name.to_csv(file_name, header=False, index=False, sep=',')
        
def create_dir(df):
    nokrow = 0
    for index, row in df.iterrows():
        
        #read cols
        user_name
        try:
            first_name = row['First Name'].lower().strip().split()[0]
            user_name += first_name
        except:
            first_name = ""
        try:            
            last_name = row['Last Name'].lower().strip().split()[0]
            user_name += '_' + last_name
        except:
            last_name = ""
        
        if len(user_name)==0:
            continue
                    
        ssh_key = row[colmap.get('ssh_key')].strip()
        
        #create folder and file
        folder_name = f'/Users/yabebal/Downloads/{prefix}-credentials/' + user_name
        file_name = folder_name + '/authorized_keys'
           
        #write 
        try:
            os.makedirs(folder_name,exist_ok=True)
            print(file_name)
            with open(file_name, 'w') as f:
                f.write(ssh_key)
            nokrow += 1
        except: # FileExistsError:
            print(f'File {file_name} already exists')
            
    print(f'Successfully completed {nokrow} outof {df.shape[0]} rows')

In [84]:
create_user_file(dfb6)
#!rm -rf /Users/yabebal/My-Works/10academy/Codes/aws_bash/compute/user/cohort-a*.txt

group G1
                user_name access
2            biniyam_ajaw   root
9   mickiyas_kinfemichael   root
10            naol_lamesa   root
11           fanuel_abebe   root
12         elias_assamnew   root
17         lillian_assefa   root
writing to file /Users/yabebal/My-Works/10academy/Codes/aws_bash/compute/user/cohort-a-w7-g1.txt
group G2
           user_name access
5      mekdes_mesfin   root
14       meron_muche   root
15       meron_muche   root
26      eyaya_eneyew   root
27  abdulhamid_mussa   root
28       meron_muche   root
32      rediet_girma   root
writing to file /Users/yabebal/My-Works/10academy/Codes/aws_bash/compute/user/cohort-a-w7-g2.txt
group G3
         user_name access
4      abel_bekele   root
22  mubarek_hussen   root
30    kerod_beyene   root
writing to file /Users/yabebal/My-Works/10academy/Codes/aws_bash/compute/user/cohort-a-w7-g3.txt
group G4
             user_name access
8    rodolphe_segbedji   root
16       diriba_bekele   root
20  alexander_mengesha  

In [42]:
create_dir(dfb6)

/Users/yabebal/Downloads/cohort-a-credentials/natnael_melese/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/biniyam_ajaw/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/nasrallah_hassan/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/abel_bekele/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/mekdes_mesfin/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/rehmet_yeshanew/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/emtinan_osman/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/rodolphe_segbedji/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/mickiyas_kinfemichael/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/naol_lamesa/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/fanuel_abebe/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/elias_assamnew/authorized_keys
/Users/yabebal/Downloads/cohort-a-credentials/fikerte_alemayheu/authorized_

In [46]:
#!aws s3 mb s3://10ac-{prefix} --profile ustenac
print(f"~/Downloads/{prefix}-credentials/ s3://10ac-{prefix}/credentials/ --recursive --profile ustenac")

~/Downloads/cohort-a-credentials/ s3://10ac-cohort-a/credentials/ --recursive --profile ustenac
